In [1]:
import pandas as pd
from match import Matcher
import enum
from copy import *
import pprint
pp = pprint.PrettyPrinter(indent=4)
import itertools
from collections import *
from tqdm import tqdm
from joblib import Parallel, delayed
from time import *

In [2]:
datafile = 'data/ay1819-usp.csv'

def get_data():
    rawdata = pd.read_csv(datafile)
    data = rawdata[['Gender Description', 'Room Preference Description 1', 'Room Preference Description 2', 'Room Preference Description 3', 'Floor Gender Preference']]
    data = data\
        .rename(columns = {
            'Gender Description': 'gender',
            'Room Preference Description 1': 'room_pref_1',
            'Room Preference Description 2': 'room_pref_2',
            'Room Preference Description 3': 'room_pref_3',
            'Floor Gender Preference': 'gender_pref',
        })\
        .replace('USP, Single (Corridor, Air-Con)', 'corri_air')\
        .replace('USP, Single (6 bdrm Apt, Air-Con)', 'suite_air')\
        .replace('USP, Single (Corridor, Non Air-Con)', 'corri_non')\
        .replace('USP, Single (6 bdrm Apt, Non Air-Con)', 'suite_non')\
        .replace('Mixed Gender Floor', 'mixed')\
        .replace('No Preference', 'mixed')\
        .replace('Single Gender Floor', 'single')\
        .fillna('no_pref')
    return data

data = get_data()

In [3]:
# Define floors
floors = set(range(4, 22))
male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = floors.difference(male_floors).difference(female_floors)
rf_floors = set(range(5, 22, 4))
laundry_floors = set([9, 17])
only_rf_floors = rf_floors.difference(laundry_floors)
normal_floors = floors.difference(rf_floors)
freshmen_floors = set([6, 8, 11, 14, 16, 17, 20])
senior_floors = floors.difference(freshmen_floors)

def get_num_floor_types_iterator():
    rf_floors_consider = only_rf_floors.difference(freshmen_floors)
    laundry_floors_consider = laundry_floors.difference(freshmen_floors)
    normal_floors_consider = normal_floors.difference(freshmen_floors)
    
    for male_rf in range(len(rf_floors_consider) + 1):
        for female_rf in range(len(rf_floors_consider) + 1 - male_rf):
            mixed_rf = len(rf_floors_consider) - male_rf - female_rf
            
            for male_laundry in range(len(laundry_floors_consider) + 1):
                for female_laundry in range(len(laundry_floors_consider) + 1 - male_laundry):
                    mixed_laundry = len(laundry_floors_consider) - male_laundry - female_laundry
            
                    for male_normal in range(len(normal_floors_consider) + 1):
                        for female_normal in range(len(normal_floors_consider) + 1 - male_normal):
                            mixed_normal = len(normal_floors_consider) - male_normal - female_normal
  
                            male = [male_rf, male_laundry, male_normal]
                            female = [female_rf, female_laundry, female_normal]
                            mixed = [mixed_rf, mixed_laundry, mixed_normal]
                            combination = [male, female, mixed]
                            yield combination

In [4]:
class RoomType(enum.Enum):
    MALE_CORRI_AIR = 0
    MALE_CORRI_NON = 1
    MALE_SUITE_AIR = 2
    MALE_SUITE_NON = 3
    FEMALE_CORRI_AIR = 4
    FEMALE_CORRI_NON = 5
    FEMALE_SUITE_AIR = 6
    FEMALE_SUITE_NON = 7
    MIXED_CORRI_AIR = 8
    MIXED_CORRI_NON = 9
    NO_PREF = 10
    NO_ROOM = 11
    
room_pref_to_type_map = {
    'Female': {
        'suite_air': RoomType.FEMALE_SUITE_AIR,
        'suite_non': RoomType.FEMALE_SUITE_NON,
        'corri_air': RoomType.FEMALE_CORRI_AIR,
        'corri_non': RoomType.FEMALE_CORRI_NON,
        'no_pref': RoomType.NO_PREF,
    },
    'Male': {
        'suite_air': RoomType.MALE_SUITE_AIR,
        'suite_non': RoomType.MALE_SUITE_NON,
        'corri_air': RoomType.MALE_CORRI_AIR,
        'corri_non': RoomType.MALE_CORRI_NON,
        'no_pref': RoomType.NO_PREF,
    },
}

mixed_room_pref_to_type = {
    'corri_air': RoomType.MIXED_CORRI_AIR,
    'corri_non': RoomType.MIXED_CORRI_NON,
    'no_pref': RoomType.NO_PREF,
}

def get_empty_room_types_count(total_num_rooms):
    room_types = Counter()
    for room_type in RoomType:
        room_types[room_type] = 0
        
    room_types[RoomType.NO_ROOM] = total_num_rooms
    return room_types

def get_floor_stats(has_rf=False, has_laundry=False):
    stats = Counter()
    stats['corri_air'] = 6
    stats['corri_non'] = 12
    stats['suite_air'] = 6
    stats['suite_non'] = 12
    
    if has_rf or has_laundry:
        stats['suite_air'] -= 6
    
    if has_laundry:
        stats['corri_non'] -= 3
        stats['suite_non'] -= 6

    return stats

In [5]:
def get_room_stats(male_floor_counts, female_floor_counts, mixed_room_counts):
    floor_counts = [male_floor_counts, female_floor_counts, mixed_room_counts]
    male_rf, male_laundry, male_normal = male_floor_counts
    male_room_stats = Counter()
    male_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(), range(male_normal))) if male_normal > 0 else Counter()
    male_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_rf=True), range(male_rf))) if male_rf > 0 else Counter()
    male_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_laundry=True), range(male_laundry))) if male_laundry > 0 else Counter()
    male_room_stats

    female_rf, female_laundry, female_normal = female_floor_counts
    female_room_stats = Counter()
    female_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(), range(female_normal))) if female_normal > 0 else Counter()
    female_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_rf=True), range(female_rf))) if female_rf > 0 else Counter()
    female_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_laundry=True), range(female_laundry))) if female_laundry > 0 else Counter()
    female_room_stats

    mixed_rf, mixed_laundry, mixed_normal = mixed_room_counts
    mixed_room_stats = Counter()
    mixed_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(), range(mixed_normal))) if mixed_normal > 0 else Counter()
    mixed_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_rf=True), range(mixed_rf))) if mixed_rf > 0 else Counter()
    mixed_room_stats += reduce(lambda x, y: x + y, map(lambda x: get_floor_stats(has_laundry=True), range(mixed_laundry))) if mixed_laundry > 0 else Counter()
    mixed_room_stats
    
    return [male_room_stats, female_room_stats, mixed_room_stats, floor_counts]

def get_room_type_count_iterator(male_room_stats, female_room_stats, mixed_room_stats, floor_counts):
    room_stats = get_empty_room_types_count(len(data))
    for room_pref in male_room_stats:
        room_type = room_pref_to_type_map['Male'][room_pref]
        room_stats[room_type] += male_room_stats[room_pref]

    for room_pref in female_room_stats:
        room_type = room_pref_to_type_map['Female'][room_pref]
        room_stats[room_type] += female_room_stats[room_pref]
    
    for room_pref in mixed_room_stats:
        if room_pref in mixed_room_pref_to_type:
            room_type = mixed_room_pref_to_type[room_pref]
            room_stats[room_type] += mixed_room_stats[room_pref]

    num_suite_air = mixed_room_stats['suite_air'] / 6
    num_suite_non = mixed_room_stats['suite_non'] / 6

    room_stats_freeze = deepcopy(room_stats)

    for num_male_suite_air in range(num_suite_air + 1):
        num_female_suite_air = num_suite_air - num_male_suite_air

        for num_male_suite_non in range(num_suite_non + 1):
            num_female_suite_non = num_suite_non - num_male_suite_non

            room_stats = deepcopy(room_stats_freeze)
            room_stats[RoomType.MALE_SUITE_AIR] += num_male_suite_air * 6
            room_stats[RoomType.MALE_SUITE_NON] += num_male_suite_non * 6
            room_stats[RoomType.FEMALE_SUITE_AIR] += num_female_suite_air * 6
            room_stats[RoomType.FEMALE_SUITE_NON] += num_female_suite_non * 6
            yield (room_stats, floor_counts)

In [6]:
def get_valid_room_types(person):
    valid_rts = []
    valid_rts.extend(room_pref_to_type_map[person.gender].values())
    if person.gender_pref == 'mixed':
        valid_rts.extend(mixed_room_pref_to_type.values())
    valid_rts.remove(RoomType.NO_PREF)
    return valid_rts

def get_mixed_or_gender_room_type(room_pref, gender):
    if room_pref in mixed_room_pref_to_type.keys():
        return mixed_room_pref_to_type[room_pref]
    
    return room_pref_to_type_map[gender][room_pref]

def get_assignment(room_type_count):
    rtc_left = deepcopy(room_type_count)
    assignments = [None for _ in range(len(data))]
    
    def assign_pref(pref_num):
        for idx, person in data.iterrows():
            if assignments[idx] is not None:
                break
                
            if pref_num == 1:
                pref = person.room_pref_1
            elif pref_num == 2:
                pref = person.room_pref_2
            elif pref_num == 3:
                pref = person.room_pref_3

            if person.gender_pref == 'mixed':
                rt = get_mixed_or_gender_room_type(pref, person.gender)

                if rt == RoomType.NO_PREF:
                    assignments[idx] = (pref_num, rt)
                    continue

                if rtc_left[rt] > 0:
                    assignments[idx] = (pref_num, rt)
                    rtc_left[rt] -= 1
                    continue

            rt = room_pref_to_type_map[person.gender][pref]
            if rt == RoomType.NO_PREF:
                    assignments[idx] = (pref_num, rt)
                    continue

            if rtc_left[rt] > 0:
                assignments[idx] = (pref_num, rt)
                rtc_left[rt] -= 1
                continue            
    
    # TODO: hard to test this cause no one has no_prefs!
    # ideally, we should see who still has a no_pref and 
    # swap with someone lower in the queue with a room
    def fix_no_prefs():
        if RoomType.NO_PREF in assignments:
            print 'no_pref_present'
        return
    
    
    def allocate_empty_rooms():
        for idx, person in data.iterrows():
            if assignments[idx] is not None:
                continue

            valid_rts = get_valid_room_types(person)

            for rt in valid_rts:
                if rtc_left[rt] > 0:
#                     print 'yay! allocated empty room'
                    assignments[idx] = ('alloc_empty', rt)
                    rtc_left[rt] -= 1
                    break
            
        return
    
    def move_mixed_to_single():
        for idx, person in data.iterrows():
            if assignments[idx] is not None:
                break
        
            if person.gender_pref == 'single':
                break
            
            for other_idx in reversed(data.index):
                if assignments[idx] is not None:
                    break

                other_gender = data.gender[other_idx]
                if other_gender == person.gender:
                    continue
                
                single_gender_rts = room_pref_to_type_map[other_gender].values()
#                 single_gender_rts.remove(RoomType.NO_PREF)
                for rt in single_gender_rts:
                    if rtc_left[rt] < 1:
                        continue
                    
                    print 'yay! swapping mixed to single'
                    assignments[idx] = assignments[other_idx]
                    assignments[other_idx] = rt
                    rtc_left[rt] -= 1
                    break                
                
    def clean_assignments():
        for i in range(len(assignments)):
            if assignments[i] is not None:
                assignments[i] = assignments[i][1]

    assign_pref(1)
    assign_pref(2)
    assign_pref(3)
    fix_no_prefs()
    allocate_empty_rooms()
    move_mixed_to_single()
    clean_assignments()
    
    return assignments

# Algo Driver

In [9]:
room_type_counts = []
floor_counts = []
fcs = list(get_num_floor_types_iterator())
for floor_count in tqdm(fcs):
    room_stats = get_room_stats(*floor_count)
    rtcs_fcs = list(get_room_type_count_iterator(*room_stats))
    rtcs, fcs = zip(*rtcs_fcs)
    room_type_counts.extend(rtcs)
    floor_counts.extend(fcs)

t = time()
all_assignments = Parallel(n_jobs=-1)(delayed(get_assignment)(rtc) for rtc in tqdm(room_type_counts))
print time() - t

print 'done'
a = all_assignments

100%|██████████| 37200/37200 [03:50<00:00, 161.25it/s]


232.072120905
done


In [13]:
none_count = map(lambda x: x.count(None), tqdm(all_assignments))
min_none_count = min(none_count)
min_none_count_index = none_count.index(min_none_count)
print min_none_count, min_none_count_index
rtc = room_type_counts[min_none_count_index]
ass = get_assignment(rtc)
rtc - Counter(ass)

100%|██████████| 37200/37200 [00:04<00:00, 8244.57it/s]

3 400


Counter({<RoomType.NO_ROOM: 11>: 366})

In [14]:
floor_counts[min_none_count_index], rtc

([[0, 0, 0], [0, 0, 2], [3, 1, 5]],
 Counter({<RoomType.FEMALE_CORRI_AIR: 4>: 12,
          <RoomType.FEMALE_CORRI_NON: 5>: 24,
          <RoomType.FEMALE_SUITE_AIR: 6>: 18,
          <RoomType.FEMALE_SUITE_NON: 7>: 54,
          <RoomType.MALE_CORRI_AIR: 0>: 0,
          <RoomType.MALE_CORRI_NON: 1>: 0,
          <RoomType.MALE_SUITE_AIR: 2>: 24,
          <RoomType.MALE_SUITE_NON: 3>: 72,
          <RoomType.MIXED_CORRI_AIR: 8>: 54,
          <RoomType.MIXED_CORRI_NON: 9>: 105,
          <RoomType.NO_PREF: 10>: 0,
          <RoomType.NO_ROOM: 11>: 366}))

In [15]:
data['val'] = map(lambda x: x if x is not None else 'empty', ass)
data[data['val'] == 'empty']

,gender,room_pref_1,room_pref_2,room_pref_3,gender_pref,val
294,Female,suite_air,suite_non,corri_non,single,empty
297,Female,suite_air,corri_non,suite_non,single,empty
356,Male,corri_air,corri_non,suite_air,single,empty


In [10]:
# # STUFF FOR STABLE MARRIAGES

# def get_mixed_or_gender_room_type(room_pref, gender):
#     if room_pref in mixed_room_pref_to_type.keys():
#         return mixed_room_pref_to_type[room_pref]
    
#     return room_pref_to_type_map[gender][room_pref]

# def get_person_room_type_rank(person):
#     ranking = []
    
#     # HANDLE MIXED GENDER PREF
#     if person.gender_pref == 'mixed':
#         if person.room_pref_1 != 'no_pref':
#             ranking.append(get_mixed_or_gender_room_type(person.room_pref_1, person.gender))
#         if person.room_pref_2 != 'no_pref':
#             ranking.append(get_mixed_or_gender_room_type(person.room_pref_2, person.gender))
#         if person.room_pref_3 != 'no_pref':
#             ranking.append(get_mixed_or_gender_room_type(person.room_pref_3, person.gender))
        
#     # HANDLE SINGLE GENDER PREF
#     if person.room_pref_1 != 'no_pref':
#         ranking.append(room_pref_to_type_map[person.gender][person.room_pref_1])

#     if person.room_pref_2 != 'no_pref':
#         ranking.append(room_pref_to_type_map[person.gender][person.room_pref_2])

#     if person.room_pref_3 != 'no_pref':
#         ranking.append(room_pref_to_type_map[person.gender][person.room_pref_3])

#     for room_pref in room_pref_to_type_map[person.gender]:
#         room_type = room_pref_to_type_map[person.gender][room_pref]
#         ranking.append(room_type)
            
#     ranking.append(RoomType.NO_ROOM)            

#     for room_type in RoomType:
#         ranking.append(room_type)
        
#     return unique_elems(ranking)
    
# def get_person_pref(person, room_types_count):
#     room_type_rank = get_person_room_type_rank(person)
#     preferences = []
#     for room_type in room_type_rank:
#         room_type_choices = map(lambda x: '{}-{}'.format(room_type.name, x), range(room_types_count[room_type]))
#         preferences.extend(room_type_choices)
    
#     return preferences

# # TODO: this is wrong!
# # consider the case where the last person's first preference is more important than the first person's second preference
# def get_room_pref(room, persons):
#     return range(len(persons))

# def get_room_type_from_room_name(room_name):
#     room_type_name, _ = room_name.split('-')
#     return RoomType[room_type_name]
    
# def get_room_type_list_from_engagements(engagements):
#     inverse_engagements = dict((v,k) for k,v in engagements.items())
#     room_type_list = []
#     for i in range(len(inverse_engagements)):
#         room_name = inverse_engagements[i]
#         room_type = get_room_type_from_room_name(room_name)
#         room_type_list.append(room_type)
    
#     return room_type_list

# room_type_list = get_room_type_list_from_engagements(engaged)

# def find_room_to_replace(person_num, room_type_list):
#     person_assignment = room_type_list[person_num]
#     if person_assignment != RoomType.NO_ROOM:
#         return -1
    
#     person = data.iloc[person_num]
#     room_types_to_consider = room_pref_to_type_map[person.gender].values()
#     if person.gender_pref == 'mixed':
#         room_types_to_consider.extend(mixed_room_pref_to_type.values())

#     for i, v in enumerate(reversed(room_type_list)):
#         replacee_room_num = len(room_type_list) - i - 1
#         if replacee_room_num <= person_num:
#             return -1
        
#         if v in room_types_to_consider:
#             return replacee_room_num
    
#     return -1

# def get_optimized_room_type_list(room_type_list):
#     rtl = deepcopy(room_type_list)
#     for replacer_num in range(len(rtl)):
#         replacee_num = find_room_to_replace(replacer_num, rtl)
#         if replacee_num != -1:
#             print replacer_num, replacee_num
#             print rtl[replacer_num], rtl[replacee_num]
#             rtl[replacer_num], rtl[replacee_num] = rtl[replacee_num], rtl[replacer_num]
    
#     return rtl

# import copy
# guyprefers = p
# galprefers = r
# guys = sorted(guyprefers.keys())
# gals = sorted(galprefers.keys())

# def check(engaged):
#     inverseengaged = dict((v,k) for k,v in engaged.items())
#     for she, he in engaged.items():
#         shelikes = galprefers[she]
#         shelikesbetter = shelikes[:shelikes.index(he)]
#         helikes = guyprefers[he]
#         helikesbetter = helikes[:helikes.index(she)]
#         for guy in shelikesbetter:
#             guysgirl = inverseengaged[guy]
#             guylikes = guyprefers[guy]
#             if guylikes.index(guysgirl) > guylikes.index(she):
#                 print("%s and %s like each other better than "
#                       "their present partners: %s and %s, respectively"
#                       % (she, guy, he, guysgirl))
#                 return False
#         for gal in helikesbetter:
#             girlsguy = engaged[gal]
#             gallikes = galprefers[gal]
#             if gallikes.index(girlsguy) > gallikes.index(he):
#                 print("%s and %s like each other better than "
#                       "their present partners: %s and %s, respectively"
#                       % (he, gal, she, girlsguy))
#                 return False
#     return True

# def matchmaker():
#     guysfree = guys[:]
#     engaged  = {}
#     guyprefers2 = copy.deepcopy(guyprefers)
#     galprefers2 = copy.deepcopy(galprefers)
#     while guysfree:
#         guy = guysfree.pop(0)
#         guyslist = guyprefers2[guy]
#         gal = guyslist.pop(0)
#         fiance = engaged.get(gal)
#         if fiance is None:
#             # She's free
#             engaged[gal] = guy
# #             print("  %s and %s" % (guy, gal))
#         else:
#             # The bounder proposes to an engaged lass!
#             galslist = galprefers2[gal]
#             if galslist.index(fiance) > galslist.index(guy):
#                 # She prefers new guy
#                 engaged[gal] = guy
# #                 print("  %s dumped %s for %s" % (gal, fiance, guy))
#                 if guyprefers2[fiance]:
#                     # Ex has more girls to try
#                     guysfree.append(fiance)
#             else:
#                 # She is faithful to old fiance
#                 if guyslist:
#                     # Look again
#                     guysfree.append(guy)
#     return engaged
 
# %time engaged = matchmaker()
# print('Engagement stability check PASSED' if check(engaged) else 'Engagement stability check FAILED')

# def solve(room_type_count):
#     all_rooms = []
#     for room_type in RoomType:
#         all_rooms.extend(map(lambda x: '{}-{}'.format(room_type.name, x), range(room_type_count[room_type])))

#     ppref = dict()
#     for idx, p in data.iterrows():
#         ppref[idx] = get_person_pref(p, room_type_count)

#     rpref = dict()
#     for room in all_rooms:
#         rpref[room] = get_room_pref(room, data)

#     m = Matcher(ppref, rpref)
#     return ppref, rpref, m